# Libraries

In [2]:
import math

# machine learning
import torch
import torch.nn as nn
import torch.nn.functional as F

%store -r features_matrix

# Probabilistic Embeddings

In [3]:
def t2v(
    tau, # input tensor
    f, # activation function (sin or cosin)
    out_features, # size of output vector
    w, # weights
    b, # biases
    w0, # weights for linear part of time2vec layer
    b0, # biases for linear part of time2vec layer
    arg=None # optional arguments
    ):
    if arg:
        v1 = f(torch.matmul(tau, w) + b, arg)
    else:
        #print(w.shape, t1.shape, b.shape)
        v1 = f(torch.matmul(tau, w) + b)
    v2 = torch.matmul(tau, w0) + b0
    #print(v1.shape)
    return torch.cat([v1, v2], -1)

class ProbabilisticSineActivation(nn.Module):
    def __init__(self, in_features, out_features):
        super(ProbabilisticSineActivation, self).__init__()
        self.out_features = out_features // 2  # Half for mean, half for variance
        self.w0 = nn.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.Parameter(torch.randn(1))
        self.w = nn.Parameter(torch.randn(in_features, self.out_features - 1))
        self.b = nn.Parameter(torch.randn(self.out_features - 1))
        self.f = torch.sin

    def forward(self, tau):
        # Calculate mean
        mean = t2v(tau, self.f, self.out_features, self.w, self.b, self.w0, self.b0)
        
        # Calculate variance (use another set of weights and biases, ensure positive variance)
        variance = F.softplus(t2v(tau, self.f, self.out_features, 
                                  torch.randn_like(self.w), torch.randn_like(self.b), 
                                  torch.randn_like(self.w0), torch.randn_like(self.b0)))
        
        return torch.cat([mean, variance], -1)

class ProbabilisticCosineActivation(nn.Module):
    def __init__(self, in_features, out_features):
        super(ProbabilisticCosineActivation, self).__init__()
        self.out_features = out_features // 2  # Half for mean, half for variance
        self.w0 = nn.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.Parameter(torch.randn(1))
        self.w = nn.Parameter(torch.randn(in_features, self.out_features - 1))
        self.b = nn.Parameter(torch.randn(self.out_features - 1))
        self.f = torch.cos

    def forward(self, tau):
        # Calculate mean
        mean = t2v(tau, self.f, self.out_features, self.w, self.b, self.w0, self.b0)
        
        # Calculate variance (use another set of weights and biases, ensure positive variance)
        variance = F.softplus(t2v(tau, self.f, self.out_features, 
                                  torch.randn_like(self.w), torch.randn_like(self.b), 
                                  torch.randn_like(self.w0), torch.randn_like(self.b0)))
        
        return torch.cat([mean, variance], -1)

In [4]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:x.size(0), :]

In [13]:
class Encoding(nn.Module):
    def __init__(self, in_features, out_features, max_len=5000):
        super(Encoding, self).__init__()
        self.time2vec = ProbabilisticSineActivation(in_features, out_features) # Or CosineActivation
        self.positional_encoding = PositionalEncoding(out_features, max_len)
        
    def forward(self, tau):
        # Compute Time2Vec embeddings
        time_embeddings = self.time2vec(tau)
        # Add positional encodings
        seq_len = tau.size(1)  # Assuming (seq_len, features) for tau
        pos_encodings = self.positional_encoding(time_embeddings).to(tau.device)
        return time_embeddings + pos_encodings[:seq_len, :]

In [14]:
# Forward pass through the model
feature_tensor = torch.tensor(features_matrix, dtype=torch.float32)

print(feature_tensor.shape)
# dimensions of the tensor -> 2 for true values and time stamps
in_features = features_matrix.shape[-1]
# length of embedding vector
out_features = 64

encoding_model = Encoding(in_features, out_features)

# check for NaN values early
if torch.isnan(feature_tensor).any():
    raise ValueError('NaN values detected in Input')

det_embeddings = encoding_model(feature_tensor)

# Check for NaN values after computation
if torch.isnan(det_embeddings).any():
    raise ValueError('NaN values detected in Embeddings')

print(det_embeddings.shape)

torch.Size([1024, 2])
torch.Size([2, 1024, 64])


In [ ]:
class Encoding(nn.Module):
    def __init__(self, in_features, out_features, max_len=5000):
        super(Encoding, self).__init__()
        # Adjustments for out_features to account for separate mean/variance in Probabilistic*Activation
        self.time2vec = ProbabilisticSineActivation(in_features, out_features // 2)  # Adjusted for mean and variance
        self.positional_encoding = PositionalEncoding(out_features // 2, max_len)  # Adjusted accordingly
        
    def forward(self, tau):
        # Generate embeddings (mean and variance separately)
        embeddings = self.time2vec(tau)  # Expecting [2, seq_len, features/2]
        
        # Apply positional encodings
        # Assuming embeddings are already separated into mean and variance
        pos_encodings = self.positional_encoding(embeddings[0]).to(tau.device)  # Apply to mean
        embeddings[0] += pos_encodings[:tau.size(1), :]
        embeddings[1] += pos_encodings[:tau.size(1), :]  # Apply to variance similarly
        
        # Assuming you handle the third component (e.g., timestamps) separately
        # final_embeddings = torch.stack([embeddings[0], embeddings[1], third_component], dim=0)
        
        return embeddings  # Adjust this based on how you incorporate the third component
